# Gesture Recognition
In this project, the attempt is to build a 3D Conv model that will be able to predict the 5 gestures correctly.

In [3]:
import numpy as np
import os

import skimage.io
import skimage.viewer
from skimage.io import imread
from skimage.transform import rescale, resize

from scipy import ndimage as ndi
from skimage import feature

from skimage.transform import rotate, AffineTransform
from skimage import transform

from skimage.util import random_noise
import random

import datetime
import os

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


We set the random seed so that the results don't vary drastically.

In [4]:
# This is used to choose between the CNN only (3D Conv) and CNN + RNN (CNN: 2D, Transfer Learning + RNN) model
# If cnn == 0, then, the CNN + RNN Model is chosen
# If cnn == 1, then the CNN only model is chosen
cnn = 0
img_nums = 15

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
np.random.seed(30)

Using TensorFlow backend.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-pa

In this block, we read the folder names for training and validation. We also set the `batch_size` here. Note that we set the batch size in such a way that we're able to use the GPU in full capacity. We keep increasing the batch size until the machine throws an error.

In [6]:
print(tf.__version__)

1.10.0


In [7]:
# 0: Left Swipe, 1: Right Swipe, 2: Stop, 3: Thumbs Down, 4: Thumbs Up
train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT//Project_data/val.csv').readlines())
train_len = len(train_doc)
val_len = len(val_doc)
train_dirs = []
val_dirs = []
for i in range(train_doc.shape[0]):
    train_dir = train_doc[i].split(';')[0]
    if(i < 100):
        val_dir = val_doc[i].split(';')[0]
        val_dirs.append(val_dir)
        train_dirs.append(train_dir)
batch_size = 90                                        #experiment with the batch size
print(f'Batch Size = {batch_size}, \nTrain length = {len(train_dirs)}, \nValidation length = {len(val_dirs)}')
print(f'Training Length = {train_len}, Validation Length = {val_len}')

Batch Size = 90, 
Train length = 100, 
Validation length = 100
Training Length = 663, Validation Length = 100


## Generator
Here, preprocessing of the images is done. Since we have images of 2 different dimensions, we resize them and create a batch of video frames. We experiment with `img_idx`, `y`,`z` and normalization such that we get high accuracy.

In [8]:
def generator(source_path, folder_list, batch_size, lenth, aug_aff = 0, aug_edg = 0, aug_nois = 0):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    if(img_nums == 30):
        img_idx = range(0, 30)                                                                     #create a list of image numbers needed for a particular video
    else:
        img_idx = range(0, 30, 2)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = lenth // batch_size                                                     # calculate the number of batches
        num_imgs = lenth - num_batches*batch_size
        for batch in range(num_batches):                                                          # we iterate over the number of batches
            x = len(img_idx)
            y = 96
            z = 96
            batch_data = np.zeros((batch_size,x,y,z,3))                                           # x is the number of images required for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5))                                               # batch_labels is the one hot representation of the output
            if(aug_aff):
                aug_batch_data = np.zeros((batch_size,x,y,z,3))
                aug_batch_labels = np.zeros((batch_size,5))
            if(aug_nois):
                nois_batch_data = np.zeros((batch_size,x,y,z,3))
                nois_batch_labels = np.zeros((batch_size,5))
            if(aug_edg):
                edg_batch_data = np.zeros((batch_size,x,y,z,3))
                edg_batch_labels = np.zeros((batch_size,5))



            for folder in range(batch_size):                                                      # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])  # read all the images in the folder
                for idx,item in enumerate(img_idx):                                               #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    image = resize(image = image, output_shape=(y, z))

                    if(aug_edg):
                        img1 = image.astype('float64')
                        img1[:, :, 0] += 0.2 * np.random.random(img1[:, :, 0].shape)
                        img1[:, :, 1] += 0.2 * np.random.random(img1[:, :, 1].shape)
                        img1[:, :, 2] += 0.2 * np.random.random(img1[:, :, 2].shape)
                        edges10 = feature.canny(img1[:, :, 0])
                        edges20 = feature.canny(img1[:, :, 0], sigma=3)
                        edges11 = feature.canny(img1[:, :, 1])
                        edges21 = feature.canny(img1[:, :, 1], sigma=3)
                        edges12 = feature.canny(img1[:, :, 2])
                        edges22 = feature.canny(img1[:, :, 2], sigma=3)
                        edg_batch_data[folder, idx, :, :, 0] = edges20
                        edg_batch_data[folder, idx, :, :, 1] = edges21
                        edg_batch_data[folder, idx, :, :, 2] = edges22

                    
                    if(aug_nois):
                        image_noise_aug = random_noise(image, var = 0.1)
                        nois_batch_data[folder, idx, :, :, 0] = image_noise_aug[:, :, 0]
                        nois_batch_data[folder, idx, :, :, 1] = image_noise_aug[:, :, 1]
                        nois_batch_data[folder, idx, :, :, 2] = image_noise_aug[:, :, 2]


                    image[:, :, 0] = (image [:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0]) - np.min(image[:, :, 0]))
                    image[:, :, 1] = (image [:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1]) - np.min(image[:, :, 1]))
                    image[:, :, 2] = (image [:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2]) - np.min(image[:, :, 2]))

                    if(aug_aff):
                        shear_val = random.uniform(0.0, -0.5)
                        tf = AffineTransform(shear = shear_val)
                        img_aff = transform.warp(image, tf, order=1, preserve_range=True, mode='wrap')
                        aug_batch_data[folder, idx, :, :, 0] = img_aff[:, :, 0]
                        aug_batch_data[folder, idx, :, :, 1] = img_aff[:, :, 1]
                        aug_batch_data[folder, idx, :, :, 2] = img_aff[:, :, 2]

                    batch_data[folder,idx,:,:,0] = image[:, :, 0]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]       #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]       #normalise and feed in the image


                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                if aug_aff:
                    aug_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

                if aug_nois:
                    nois_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    
                if(aug_edg):
                    edg_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            if aug_aff:
                batch_data = np.append(batch_data, aug_batch_data, axis = 0)
                batch_labels = np.append(batch_labels, aug_batch_labels, axis = 0) 

            if aug_nois:
                batch_data = np.append(batch_data, nois_batch_data, axis = 0) 
                batch_labels = np.append(batch_labels, nois_batch_labels, axis = 0)
                
            if aug_edg:
                batch_data = np.append(batch_data, edg_batch_data, axis = 0) 
                batch_labels = np.append(batch_labels, edg_batch_labels, axis = 0) 



            yield batch_data, batch_labels # yield the batch_data and the batch_labels, remember what does yield do


        # code for the remaining data points which are left after full batches
        x = len(img_idx)
        y = 96
        z = 96
        batch_data = np.zeros((num_imgs,x,y,z,3))                                               # x is the number of images used for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((num_imgs,5))

        if(aug_aff):
            aug_batch_data = np.zeros((num_imgs,x,y,z,3))
            aug_batch_labels = np.zeros((num_imgs,5))
        if(aug_nois):
            nois_batch_data = np.zeros((num_imgs,x,y,z,3))
            nois_batch_labels = np.zeros((num_imgs,5))
        if(aug_edg):
            edg_batch_data = np.zeros((num_imgs,x,y,z,3))
            edg_batch_labels = np.zeros((num_imgs,5))

        for folder in range(num_imgs):
          imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
          for idx, item in enumerate(img_idx):
            image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            #crop the images and resize them. Note that the images are of 2 different shapes 
            #and the conv3D will throw error if the inputs in a batch have different shapes

            image = resize(image = image, output_shape=(y, z))

            if(aug_edg):
                img1 = image.astype('float64')
                img1[:, :, 1] += 0.2 * np.random.random(img1[:, :, 1].shape)
                img1[:, :, 2] += 0.2 * np.random.random(img1[:, :, 2].shape)
                edges10 = feature.canny(img1[:, :, 0])
                edges20 = feature.canny(img1[:, :, 0], sigma=3)
                edges11 = feature.canny(img1[:, :, 1])
                edges21 = feature.canny(img1[:, :, 1], sigma=3)
                edges12 = feature.canny(img1[:, :, 2])
                edges22 = feature.canny(img1[:, :, 2], sigma=3)
                edg_batch_data[folder, idx, :, :, 0] = edges20
                edg_batch_data[folder, idx, :, :, 1] = edges21
                edg_batch_data[folder, idx, :, :, 2] = edges22

                    
            if(aug_nois):
                image_noise_aug = random_noise(image, var = 0.1)
                nois_batch_data[folder, idx, :, :, 0] = image_noise_aug[:, :, 0]
                nois_batch_data[folder, idx, :, :, 1] = image_noise_aug[:, :, 1]
                nois_batch_data[folder, idx, :, :, 2] = image_noise_aug[:, :, 2]

            image[:, :, 0] = (image [:, :, 0] - np.min(image[:, :, 0]))/(np.max(image[:, :, 0]) - np.min(image[:, :, 0]))
            image[:, :, 1] = (image [:, :, 1] - np.min(image[:, :, 1]))/(np.max(image[:, :, 1]) - np.min(image[:, :, 1]))
            image[:, :, 2] = (image [:, :, 2] - np.min(image[:, :, 2]))/(np.max(image[:, :, 2]) - np.min(image[:, :, 2]))

            if(aug_aff):
                shear_val = random.uniform(0.0, -0.5)
                tf = AffineTransform(shear = shear_val)
                img_aff = transform.warp(image, tf, order=1, preserve_range=True, mode='wrap')
                aug_batch_data[folder, idx, :, :, 0] = img_aff[:, :, 0]
                aug_batch_data[folder, idx, :, :, 1] = img_aff[:, :, 1]
                aug_batch_data[folder, idx, :, :, 2] = img_aff[:, :, 2]
                
            batch_data[folder,idx,:,:,0] = image [:, :, 0]
            batch_data[folder,idx,:,:,1] = image [:, :, 1]
            batch_data[folder,idx,:,:,2] = image [:, :, 2]


            batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_aff:
                aug_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_nois:
                nois_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            if aug_edg:
                edg_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1


                
        if aug_aff:
            batch_data = np.append(batch_data, aug_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, aug_batch_labels, axis = 0) 

        if aug_nois:
            batch_data = np.append(batch_data, nois_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, nois_batch_labels, axis = 0) 

        if aug_edg:
            batch_data = np.append(batch_data, edg_batch_data, axis = 0)
            batch_labels = np.append(batch_labels, edg_batch_labels, axis = 0) 

            
        yield batch_data, batch_labels                                                            # yield the batch_data and the batch_labels, remember what does yield do

We note, here, that a video is represented above in the generator as (number of images, height, width, number of channels). We take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'               # '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'                   # '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15                                                          # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here we make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. We use `TimeDistributed` while building a Conv2D + RNN model. Also, the last layer is the softmax. We design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [11]:
import tensorflow as tf

if(cnn == 0):
    input_shape = (96, 96, 3)

    mobv2_model = keras.applications.MobileNetV2(input_shape=input_shape, weights = 'imagenet', include_top=False)

    mobv2_model.trainable = False
#     num_layers = len(mobv2_model.layers)     
#     print(num_layers)
    
#     layer_split = 4*num_layers//5
#     print(layer_split)
#     for layer in range(layer_split):
#         mobv2_model.layers[layer].trainable = False
#     for layer in range(layer_split, num_layers):
#         mobv2_model.layers[layer].trainable = True
        
    cnn_model = Sequential([
        mobv2_model,
        keras.layers.Flatten(),
        keras.layers.Dense(64),
#        keras.layers.BatchNormalization(),
#        keras.layers.Dropout(0.25)
    ])
    cnn_model.summary()

9412608/9406464 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
flatten_1 (Flatten)          (None, 11520)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                737344    
Total params: 2,995,328
Trainable params: 737,344
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
# from keras.layers import Lambda
from keras import regularizers

def rnn_gru(input_dims, n_classes, cnn_model):
    model = Sequential()
    model.add(TimeDistributed(cnn_model, input_shape = input_dims))
#    model.add(GRU(64, input_shape = input_dims, return_sequences=False, kernel_regularizer = regularizers.l2(1e-02), dropout = 0.25))
#    model.add(GRU(64, input_shape = input_dims, return_sequences=False, dropout = 0.25))
    model.add(GRU(64, input_shape = input_dims, return_sequences=False))

    model.add(Dense(64, activation = 'relu'))
#    model.add(BatchNormalization())
#    model.add(Dropout(0.25))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [13]:
if(cnn == 0):
    if(img_nums == 30):
        input_dimens = (30, 96, 96, 3)
    else:
        input_dimens = (15, 96, 96, 3)
    model = rnn_gru(input_dims = input_dimens, n_classes = 5, cnn_model = cnn_model)

Now that, having written the model, the next step is to `compile` the model. When we print the `summary` of the model, we see the total number of parameters we need to train.

In [14]:
if(cnn == 0):
    #optimiser = keras.optimizers.Adam(learning_rate = 0.01)                            # optimizer
    #optimiser = 'Adam'
    #model.compile(optimizer=keras.optimizers.Adam(lr = 0.1), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    #model.compile(optimizer=tf.train.AdamOptimizer(0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    optimiser = 'Adam'
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


    print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 64)            2995328   
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 3,024,581
Trainable params: 766,597
Non-trainable params: 2,257,984
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
if(cnn == 0):
    aug_aff = 0
    aug_edg = 0
    aug_nois = 0
    train_generator = generator(train_path, train_doc, batch_size, train_len, aug_aff = 1, aug_edg = 0, aug_nois = 1)
    val_generator = generator(val_path, val_doc, batch_size, val_len, aug_aff = 0, aug_edg = 0, aug_nois = 0)
    print(train_generator)
    print(val_generator)

<generator object generator at 0x7fc6b7505eb8>
<generator object generator at 0x7fc6b7505e08>


In [16]:
if(cnn == 0):
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)     # write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(f'Steps per epoch = {steps_per_epoch}, validation_steps = {validation_steps}')

Steps per epoch = 8, validation_steps = 2


Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the model at the end of each epoch.

In [18]:
if(cnn == 0):
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 90
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 90
Epoch 1/15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


8/8 [==============================] - 216s 27s/step - loss: 1.4283 - categorical_accuracy: 0.3876 - val_loss: 1.0427 - val_categorical_accuracy: 0.6800

Epoch 00001: saving model to model_init_2021-02-0817_28_14.929770/model-00001-1.43587-0.38411-1.04267-0.68000.h5
Epoch 2/15
8/8 [==============================] - 97s 12s/step - loss: 1.0128 - categorical_accuracy: 0.6020 - val_loss: 0.7658 - val_categorical_accuracy: 0.7100

Epoch 00002: saving model to model_init_2021-02-0817_28_14.929770/model-00002-1.02888-0.59477-0.76575-0.71000.h5
Epoch 3/15
8/8 [==============================] - 104s 13s/step - loss: 0.8289 - categorical_accuracy: 0.6987 - val_loss: 0.8330 - val_categorical_accuracy: 0.6600

Epoch 00003: saving model to model_init_2021-02-0817_28_14.929770/model-00003-0.83415-0.69734-0.83300-0.66000.h5
Epoch 4/15
8/8 [==============================] - 101s 13s/step - loss: 0.6970 - categorical_accuracy: 0.7406 - val_loss: 0.6347 - val_categorical_accuracy: 0.7600

Epoch 00004: 